<center>

# **如何运行Heudiconv**

</center>

***


Python神经影像社区日趋完善，Linux shell搭配Python的神经影像工具开发方式越来越受追捧，催生出fMRIPrep、fMRIDenoise、Nipype、Nilearn、PyMVPA等优秀的toolbox，今天给大家介绍一个前期准备数据，自动整理BIDS格式的软件。

基于Linux及MacOS的Heudiconv是一款很方便的dicom转nifti格式的工具，不同于dcm2nii等传统软件仅提供格式转换，Heudiconv的**亮点**在于自动组织图像路径，生成结构化的影像数据存储模式——BIDS格式,详细信息可查看[heudiconv 官方文档](https://neuroimaging-core-docs.readthedocs.io/en/latest/pages/heudiconv.html)。

本教程适合 **零基础入门** 的同学，带你一步步理解并使用 HeuDiConv 工具完成 DICOM → BIDS 的格式转换。

在阅读和运行代码之前，你需要理解以下几点：


- **DICOM**：医学影像的原始格式（来自扫描仪）。
- **BIDS (Brain Imaging Data Structure)**：神经影像数据的国际标准化格式，方便共享、预处理和分析。
- **HeuDiConv**：一个自动化工具，帮助我们从 DICOM 转换到 BIDS。

**步骤：**

1. 搭建运行环境
2. 使用 HeuDiConv 获取 dicominfo.tsv
3. 根据 dicominfo 生成 heuristic.py
4. 使用 heuristic.py 转换并验证 BIDS 是否合规
5. 逐步处理 ses-005、ses-025 等多个 session，重复上述步骤直到数据集完整


## 设置运行环境

In [ ]:
# 通过 Docker 拉取 HeuDiConv 镜像
docker pull nipy/heudiconv:latest

注：进行docker pull 不成功，并且是因为网络问题的时候，可参考以下链接：

- https://gitee.com/wanfeng789/docker-hub#docker-hub
- https://www.bilibili.com/video/BV1Zn4y19743/ 
- https://status.1panel.top/status/docker

## 运行HeuDiConv

首先，我们需要将自己的Dicom格式文件整理好，建议存放的路径为：

项目文件夹 -> Dicom数据文件夹 -> 被试文件夹 -> session文件夹 (optional) -> 各模态文件夹 -> 具体的 .dcm 文件

**注：**session一般指研究里大部分被试参与了的两次及以上相同扫描序列的扫描，纵向研究中涉及较多，如治疗前后，或者纵向多时间点随访等，如果你的数据涉及可以加这一级文件夹，如果不涉及则可以忽略这一级。

例如：

```
/home/ss/Downloads/HeuDiConv_walkthrough/Dicom/sub-01/ses-001/SCANS/
   ├── 3/DICOM/*.dcm
   ├── 4/DICOM/*.dcm
   ├── 5/DICOM/*.dcm
   ├── 6/DICOM/*.dcm
   ├── 7/DICOM/*.dcm
   ├── 8/DICOM/*.dcm
   └── 9/DICOM/*.dcm

```

In [ ]:
# 在终端执行以下命令，记得替换路径

docker run --rm -it \ 
  -v /你的路径/HeuDiConv_walkthrough:/base \
  nipy/heudiconv:latest \
  -d /base/Dicom/sub-{subject}/ses-{session}/SCANS/*/DICOM/*.dcm \
  -o /base/Nifti/ \
  -f convertall \
  -s 01 \
  -ss 001 \
  -c none \
  --overwrite

### 相关参数解释


| 参数 | 说明 | 示例 |
|------|------|------|
| `docker run --rm -it` | 启动一个 Docker 容器，容器运行后自动删除，交互模式 |/|
| `-v` | 把本地路径挂载到容器内部 |/|
| `nipy/heudiconv:latest` | 使用的 Docker 镜像名 |/|
| `-d` | 输入的路径，从`/base`开始，使用`*`表示不同的protocols | `/base/Dicom/sub-{subject}/ses-{session}/SCANS/*/DICOM/*.dcm` |
| `-o` | 输出目录 | `/base/Nifti/` |
| `-f` | heuristic 文件（命名规则） | convertall(HeuDiConv 内置的默认规则，会把所有序列都转换，不做特别筛选) ，来转换`.tsv`文件并读取原始信息|
| `-s` | 指定被试编号 | `01` (处理 sub-01) |
| `-ss` | 指定session编号| `001` (对应 ses-001) |
| `-c` | 选择转换工具 | none (只生成 BIDS 结构的 JSON 信息，不实际进行数据转换)|
| `--overwrite` | 覆盖已存在的输出结果 | - |


使用Heudiconv读取DICOM信息后，在输出的 `Nifti` 文件夹中会生成`.heudiconv`隐藏文件夹，保存了基本的DICOM信息（例如`.tsv`文件）。可以将该文件复制出来，用于编写Python脚本来自动化批处理。

## 提取 dicominfo.tsv 并编写 heuristic.py

将生成的 dicominfo_ses-001.tsv 文件复制出来（例如放在项目根目录）：

In [ ]:
# 记得替换路径
cp /你的路径/HeuDiConv_walkthrough/Nifti/.heudiconv/01/info/dicominfo_ses-001.tsv /你的路径/HeuDiConv_walkthrough/


打开这个文件，查看其中多个参数字段（如 SeriesDescription、ProtocolName 等），用于生成你的 heuristic.py 文件。


若使用本教程的示例教程，可参考本教程的 `heuristic.py`，该脚本定义了各模态（如功能性影像、结构影像等）对应的输出路径和文件名，并通过`for`循环根据`.tsv`中的标签信息来确定每个扫描protocols的唯一性。

注：本教程的heuristic.py放在 `HeuDiConv_walkthrough` 文件夹的根目录下


**那么我们为什么需要自己编写 heuristic.py 呢？**

在第一阶段使用 `-f convertall` 时，HeuDiConv 会使用内置的默认规则，`convertall` 会转换所有找到的 DICOM 序列，包括可能不需要的序列；第二，自动生成的文件名可能不符合 BIDS 标准，导致后续分析工具无法正确识别，第三，无法根据具体的扫描协议将不同模态的数据正确分类到对应的 BIDS 子目录。

因此我们需要编写自定义的 `heuristic.py` 文件，从而做到：

- **精确控制**：只转换需要的扫描序列
- **标准命名**：确保输出文件名完全符合 BIDS 规范
- **正确分类**：将 T1、T2、fMRI、DTI 等不同模态数据放入正确的目录
- **元数据管理**：为每种数据类型添加适当的 JSON 元数据

In [ ]:
# 示例 heuristic.py 文件 （基于公开数据： MyConnectome dataset）

def create_key(template, outtype=('nii.gz',), annotation_classes=None):
    return template, outtype, annotation_classes

def infotodict(seqinfo):
    """Heuristic evaluator for determining which runs belong where"""
    info = {}

    
    t1w = create_key('sub-{subject}/{session}/anat/sub-{subject}_{session}_T1w')

    info[t1w] = []

    for s in seqinfo:
        desc = s.series_description.lower()
        if "t1" in desc or "mprage" in desc:
            info[t1w].append(s.series_id)

    return info


## 使用 heuristic.py 再次运行 HeuDiConv 并验证

使用包含自定义规则的 `heuristic.py` 再次运行转换命令，基本结构与上次相同，但加入 `-f convertall` 要修改为 `-f heuristic.py` 参数。

In [ ]:
sudo docker run --rm -it \
-v /home/ss/Downloads/HeuDiConv_walkthrough:/base \
nipy/heudiconv:latest \
-d /base/Dicom/sub-{subject}/ses-{session}/SCANS/*/DICOM/*.dcm \
-o /base/Nifti/ \
-f /base/heuristic.py \
-s 01 \
-ss 001 \
-c dcm2niix \
--overwrite


完成后，可使用[BIDS验证工具](http://bids-standard.github.io/bids-validator/)（支持Firefox和Chrome浏览器）来验证数据是否符合BIDS标准。

在本教程中，我们仅示范了一个session的数据转换过程，对于其他的session 可以同样执行上述代码，修改 docker 执行命令中的对应参数即可。

当然也可以使用脚本来批量转换所有被试，可以通过使用`for`循环来批量转换所有被试。